In [48]:
#
import requests
import json
import time
from time import sleep
import math

CONNECTED_NODE_ADDRESS = "http://127.0.0.1:"
NODE = ["http://localhost:5000", "http://localhost:5001"]

def new_transaction(port, sender, recipient, amount):
    addr = CONNECTED_NODE_ADDRESS+repr(port)
    print(addr)
    new_tx_address = "{}/transactions/new".format(addr)
    ts = time.time()
    data = {'ts' : ts,
            'sender': sender,
            'recipient': recipient,
            'amount': amount}

    response = requests.post(new_tx_address, json=data, headers={'Content-type': 'application/json'})
    if response.status_code == 201:
        print(response.content)

def initialize(port, overlap):
    url = f"http://localhost:{port}/setoverlap"
    data = {"overlap":overlap}
    response = requests.post(url,json=data, headers={"Content-Type":'application/json'})
    if response.status_code == 200:
        print(response.content)
        
def print_worldstate(node):
    print("http://localhost:{node}/printworldstate")
    response = requests.get(f"http://localhost:{node}/printworldstate")
    print(response.content)


def register_to_anchor(anchor, node):
    data = {'node_address': f'http://localhost:{anchor}'}
    register_address = f'http://localhost:{node}/register_with'
    response = requests.post(register_address, json=data, headers={"Content-Type": 'application/json'})
    if response.status_code ==200:
        print(json.loads(response.content))


def querybalance(key,n):
    file = open("query_latency",'a')
    url = "{}/query".format(NODE[0])
    data = {"key":key}
    start = time.time()
    response = requests.post(url,json=data, headers={"Content-Type":'application/json'})
    end = time.time()
    elapsed = end-start
    file.write(f'{n} {elapsed}\n')
    file.close()
    if response.status_code == 200:
        print(response.content)

def wholeshardquery(key, n):
    file = open("query_latency",'a')
    url = "{}/wholeshardquery".format(NODE[0])
    data = {"sender":key}
    start = time.time()
    response = requests.post(url,json=data, headers={"Content-Type":'application/json'})
    end = time.time()
    elapsed = end-start
    file.write(f'{n} {elapsed}\n')
    file.close()
    #data = json.loads(response.content)
    #print(json.dumps(data, indent=4, sort_keys=True))
    return

def shardinit():
    response=requests.get(CONNECTED_NODE_ADDRESS+"5000/shardinit")
    print(response.content)

def printchain(port):
    url = "{}/printchain".format(CONNECTED_NODE_ADDRESS+repr(port))
    response = requests.get(url)
    print(response.content)

def print_tracker(port):
    url = "{}/printtracker".format(CONNECTED_NODE_ADDRESS+repr(port))
    response = requests.get(url)
    print(response.content)
    
def shardedchain(port):
    url = f'{CONNECTED_NODE_ADDRESS +repr(port)}/shardedchain/5003'
    response = requests.get(url)
    print(response.content)
    
def getsize(port):
    url = f'{CONNECTED_NODE_ADDRESS+repr(port)}/getsize'
    response= requests.get(url)
    print(response.content)
    

In [12]:
#
for i in range(4):
   new_transaction(5000)

register_to_anchor(5000,5001)
shardinit()

for i in range(4):
    new_transaction(5000)

shardinit()

printchain(5000)
printchain(5001)

http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
Registration successful
b'init shard returned'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
b'init shard returned'
b'print chain'
b'print chain'


In [9]:
print_tracker(5000)
print_tracker(5001)
print_tracker(5002)
print_tracker(5003)

b'print tracker'
b'print tracker'
b'print tracker'


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5003): Max retries exceeded with url: /printtracker (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fca1eb2e190>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [45]:
printchain(5000)
printchain(5001)
printchain(5002)
printchain(5003)

b'print chain'
b'print chain'
b'print chain'
b'print chain'


In [15]:
print_worldstate(5000)
print_worldstate(5001)
print_worldstate(5002)
print_worldstate(5003)

http://localhost:{node}/printworldstate
b'print worldstate'
http://localhost:{node}/printworldstate
b'print worldstate'
http://localhost:{node}/printworldstate
b'print worldstate'
http://localhost:{node}/printworldstate
b'print worldstate'


In [48]:
#benchmark chain size estimation
peer=list(range(5000,5015))
m = len(peer)+1
for n in range(1,m):
    for i in range(100):
        new_transaction(5000, 'A','B',1)

    for p in peer:
        if p != 5000:
            register_to_anchor(5000,p)

    sleep(5)
    shardinit()
    sleep(5)
    for p in peer: 
        getsize(p)
        sleep(2)

    
    for p in peer:
        initialize(p,n+1)
    sleep(1)

http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0

b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has 

b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
Registration successful
Registration successful
Registration successful
Registration successful
Registration successful
Registration successful
Registration successful
Registration successful
Registration successful
Registration successful
Registration successful
Registration successful
Registration successful
Registration successful
b'init shard returned'
b'get size function returned'
b'get size function returned'
b'get size function returned'
b'get size function returned'
b'get size function ret

b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has 

b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has 

...

b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has 

b'get size function returned'
b'get size function returned'
b'get size function returned'
b'get size function returned'
b'get size function returned'
b'get size function returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://

b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has 

b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has 

b'get size function returned'
b'get size function returned'
b'get size function returned'
b'get size function returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'
b'setoverlap returned'


In [52]:
peer=list(range(5000,5015))
m = len(peer)+1

for n in range(1,m):
    for i in range(400):
        new_transaction(5000,'A','B',1)

    for p in peer:
        if p != 5000:
            register_to_anchor(5000,p)
            
    shardinit()
    sleep(2)
    
    for k in range(10):
        wholeshardquery('A',n)
    
    for p in peer:
        initialize(p,n+1)
    sleep(1)
    print("value of n: ",n)

http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0

b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has 

b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has 

b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has 

b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has 

...

b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has 

b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has 

b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has 

b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has 

b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has been broadcasted'
http://127.0.0.1:5000
b'block has 

In [ ]:
for i in range(100):
        new_transaction(5000,'A','B',1)

    for p in peer:
        if p != 5000:
            register_to_anchor(5000,p)
            
    shardinit()
    sleep(2)
    
    for k in range(10):
        wholeshardquery('A',n)
    